# Analysis Development
 - Given the user has access to Spotify, this script will return a simple plot of their liked songs grouped by day and number of songs liked per day
 - Additional updated research will be appended in here

##### Import the modules

In [3]:
import numpy as np
import pandas as pd

import spotipy

##### Client ID and Secret are passed from an external source